<h3>Descritization</h3>

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [12]:
df = pd.read_csv("train.csv",usecols=["Age","Fare","Survived"])
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [13]:
df.shape

(891, 3)

In [14]:
df.isnull().sum()

Survived      0
Age         177
Fare          0
dtype: int64

In [15]:
df.dropna(inplace=True)

In [17]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0])

In [18]:
X_train.shape,X_test.shape

((535, 2), (179, 2))

In [19]:
clf = DecisionTreeClassifier()

In [20]:
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
accuracy_score(y_pred,y_test)

0.659217877094972

In [21]:
#cross check with cross val score
np.mean(cross_val_score(DecisionTreeClassifier(),X_train,y_train,scoring="accuracy",cv = 10))

0.6356743535988818

In [22]:
kbin_age = KBinsDiscretizer(n_bins=5,encode="ordinal",strategy="kmeans")
kbin_fare = KBinsDiscretizer(n_bins=5,encode="ordinal",strategy="kmeans")

In [25]:
X_train.head()

,Age,Fare
313,28.0,7.8958
110,47.0,52.0000
210,24.0,7.0500
218,32.0,76.2917
881,33.0,7.8958


In [26]:
trf = ColumnTransformer([
    ("first",kbin_age,[0]),
    ("second",kbin_fare,[1])
])

In [29]:
X_train_transformed = trf.fit_transform(X_train)
X_test_transformed = trf.transform(X_test)

c:\Users\HEET BHATT\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_discretization.py:239: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
c:\Users\HEET BHATT\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_discretization.py:239: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


In [31]:
trf.named_transformers_

{'first': KBinsDiscretizer(encode='ordinal', strategy='kmeans'),
 'second': KBinsDiscretizer(encode='ordinal', strategy='kmeans')}

In [33]:
trf.named_transformers_["first"].n_bins

5

In [35]:
trf.named_transformers_["second"].bin_edges_

array([array([  0.        ,  41.91731931, 101.32966167, 187.32844231,
              376.305225  , 512.3292    ])                           ],
      dtype=object)

In [37]:
output = pd.DataFrame({"age":X_train["Age"], "age_transformed":X_train_transformed[:,0],"fare":X_train["Fare"],"fare_transformed":X_train_transformed[:,1]})

In [38]:
output['age_labels'] = pd.cut(x=X_train['Age'],
                                    bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output['fare_labels'] = pd.cut(x=X_train['Fare'],
                                    bins=trf.named_transformers_['second'].bin_edges_[0].tolist())

In [39]:
output.head()

,age,age_transformed,fare,fare_transformed,age_labels,fare_labels
313,28.0,2.0,7.8958,0.0,"(27.198, 40.025]","(0.0, 41.917]"
110,47.0,3.0,52.0000,1.0,"(40.025, 54.584]","(41.917, 101.33]"
210,24.0,1.0,7.0500,0.0,"(13.004, 27.198]","(0.0, 41.917]"
218,32.0,2.0,76.2917,1.0,"(27.198, 40.025]","(41.917, 101.33]"
881,33.0,2.0,7.8958,0.0,"(27.198, 40.025]","(0.0, 41.917]"


In [40]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)
y_pred=clf.predict(X_test_transformed)
accuracy_score(y_pred,y_test)

0.6815642458100558